# Pandas groupby 集計 高速化

[Pandas groupby 集計を10倍高速化する3つの基本テクニック](https://www.salesanalytics.co.jp/datascience/datascience268/)


In [1]:
import pandas as pd

In [2]:
import pandas as pd
# まずは6行だけの小さなデータで試してみます
df_sample = pd.DataFrame({
    'region':  ['関東', '関東', '関西', '関西', '関西', '中部'],
    'channel': ['店舗', 'オンライン', '店舗', 'オンライン', '店舗', 'オンライン'],
    'sales':   [100, 200, 150, 80, 120, 90]
})
# データの中身を確認
print(df_sample)
 

  region channel  sales
0     関東      店舗    100
1     関東   オンライン    200
2     関西      店舗    150
3     関西   オンライン     80
4     関西      店舗    120
5     中部   オンライン     90


In [3]:
# 地域ごとの売上合計を計算
result = df_sample.groupby('region')['sales'].sum()
print(result)

region
中部     90
関東    300
関西    350
Name: sales, dtype: int64


## 複数の条件でグループ化する

In [4]:
# 地域×チャネル別の平均売上を計算
# as_index=Falseを付けると、結果が見やすい表形式になります
result = df_sample.groupby(['region', 'channel'], as_index=False)['sales'].mean()
print(result)

  region channel  sales
0     中部   オンライン   90.0
1     関東   オンライン  200.0
2     関東      店舗  100.0
3     関西   オンライン   80.0
4     関西      店舗  135.0


## 高速化

### 検証用データの準備

In [5]:
import numpy as np
import pandas as pd
import time
# 乱数のシードを固定
np.random.seed(42)
# 100万行のデータを生成
N = 1_000_000  # アンダースコアで区切ると読みやすい
regions = ["関東", "関西", "中部", "東北", "九州"]
channels = ["オンライン", "店舗", "パートナー", "電話"]
df = pd.DataFrame({
    "region": np.random.choice(regions, size=N),    # 地域をランダムに選択
    "channel": np.random.choice(channels, size=N),   # チャネルをランダムに選択
    "sales": np.random.normal(500000, 10000, size=N).round(3)  # 平均500000、標準偏差10000の正規分布
})
print(f"データのサイズ: {len(df):,}行")  # カンマ区切りで表示
print(df.head())  # 最初の5行を確認

データのサイズ: 1,000,000行
  region channel       sales
0     東北   オンライン  497092.948
1     九州   パートナー  482762.557
2     中部      店舗  497356.177
3     九州      店舗  494995.646
4     九州      電話  490321.006


### 最適化前の処理速度を測定

In [6]:
# データ型を確認（regionとchannelがobject型になっているはず）
print("現在のデータ型:")
print(df.dtypes)
print()
# メモリ使用量を確認
memory_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f"メモリ使用量: {memory_mb:.2f} MB")
print()
# 速度を表示
start_time = time.perf_counter()  # 正確な時間測定用のタイマーを開始
result_before = df.groupby(["region", "channel"])['sales'].agg(['sum', 'mean', 'count'])
time_before = time.perf_counter() - start_time
print(f"処理時間（最適化前）: {time_before:.3f} 秒")

現在のデータ型:
region      object
channel     object
sales      float64
dtype: object

メモリ使用量: 174.52 MB

処理時間（最適化前）: 0.120 秒


### データ型を最適化して再測定

* カテゴリ型
* 実際に存在するグループだけを処理
* 並べ替えをスキップ

In [7]:
# データをコピー（元のデータは残しておく）
df_optimized = df.copy()
# カテゴリ型に変換
df_optimized["region"] = df_optimized["region"].astype("category")
df_optimized["channel"] = df_optimized["channel"].astype("category")
print("最適化後のデータ型:")
print(df_optimized.dtypes)
print()
# メモリ使用量を再確認
memory_mb_optimized = df_optimized.memory_usage(deep=True).sum() / 1024**2
print(f"メモリ使用量: {memory_mb_optimized:.2f} MB")
print(f"メモリ削減率: {(1 - memory_mb_optimized/memory_mb)*100:.1f}%")
print()
# 最適化された処理の実行
start_time = time.perf_counter()
result_after = df_optimized.groupby(
    ["region", "channel"], 
    observed=True,  # 実際に存在するグループだけを処理
    sort=False      # 並べ替えをスキップ
)['sales'].agg(['sum', 'mean', 'count'])
# 速度を表示
time_after = time.perf_counter() - start_time
print(f"処理時間（最適化後）: {time_after:.3f} 秒")
print(f"高速化率: {time_before / time_after:.1f}倍")

最適化後のデータ型:
region     category
channel    category
sales       float64
dtype: object

メモリ使用量: 9.54 MB
メモリ削減率: 94.5%

処理時間（最適化後）: 0.033 秒
高速化率: 3.6倍


## 保存

In [8]:
# デモ用にCSVファイルを作成
df.to_csv("sales_data.csv", index=False)

## チャンク処理

In [9]:
"""大規模CSVファイルを少しずつ読み込んで集計する関数"""
def process_large_csv(filename, chunksize=10_000):
    
    print(f"ファイル '{filename}' を {chunksize:,}行ずつ処理します...")
    
    # 集計結果を保存する変数（最初は空）
    aggregated = None
    chunk_count = 0
    
    # ファイルを少しずつ読み込む
    for chunk in pd.read_csv(filename, chunksize=chunksize):
        chunk_count += 1
        print(f"  チャンク {chunk_count} を処理中...")
        
        # 各チャンクでカテゴリ型に変換
        chunk["region"] = chunk["region"].astype("category")
        chunk["channel"] = chunk["channel"].astype("category")
        
        # チャンクごとに集計
        chunk_result = chunk.groupby(
            ["region", "channel"], 
            observed=True
        )["sales"].sum()
        
        # 結果を累積していく
        if aggregated is None:
            # 最初のチャンクの結果をそのまま保存
            aggregated = chunk_result
        else:
            # 2番目以降は既存の結果に加算
            # fill_value=0 により、片方にしかないキーも0として扱われる
            aggregated = aggregated.add(chunk_result, fill_value=0)
    
    print(f"処理完了！合計 {chunk_count} チャンクを処理しました。")
    return aggregated
# チャンク処理を実行
result = process_large_csv("sales_data.csv", chunksize=20_000)
print("\n集計結果（上位5件）:")
print(result.head())

ファイル 'sales_data.csv' を 20,000行ずつ処理します...
  チャンク 1 を処理中...
  チャンク 2 を処理中...
  チャンク 3 を処理中...
  チャンク 4 を処理中...
  チャンク 5 を処理中...
  チャンク 6 を処理中...
  チャンク 7 を処理中...
  チャンク 8 を処理中...
  チャンク 9 を処理中...
  チャンク 10 を処理中...
  チャンク 11 を処理中...
  チャンク 12 を処理中...
  チャンク 13 を処理中...
  チャンク 14 を処理中...
  チャンク 15 を処理中...
  チャンク 16 を処理中...
  チャンク 17 を処理中...
  チャンク 18 を処理中...
  チャンク 19 を処理中...
  チャンク 20 を処理中...
  チャンク 21 を処理中...
  チャンク 22 を処理中...
  チャンク 23 を処理中...
  チャンク 24 を処理中...
  チャンク 25 を処理中...
  チャンク 26 を処理中...
  チャンク 27 を処理中...
  チャンク 28 を処理中...
  チャンク 29 を処理中...
  チャンク 30 を処理中...
  チャンク 31 を処理中...
  チャンク 32 を処理中...
  チャンク 33 を処理中...
  チャンク 34 を処理中...
  チャンク 35 を処理中...
  チャンク 36 を処理中...
  チャンク 37 を処理中...
  チャンク 38 を処理中...
  チャンク 39 を処理中...
  チャンク 40 を処理中...
  チャンク 41 を処理中...
  チャンク 42 を処理中...
  チャンク 43 を処理中...
  チャンク 44 を処理中...
  チャンク 45 を処理中...
  チャンク 46 を処理中...
  チャンク 47 を処理中...
  チャンク 48 を処理中...
  チャンク 49 を処理中...
  チャンク 50 を処理中...
処理完了！合計 50 チャンクを処理しました。

集計結果（上位5件）:
region  channel
中部      オンライン 

## ベクトル化

### ベクトル化しない

In [20]:
# 売上が1200円を超える割合を地域ごとに計算したい

# データの準備
N = 10_000_000
df_large = pd.DataFrame({
    "region": np.random.choice(regions, size=N),
    "sales": np.random.normal(1000, 200, size=N).round(2)
})
# apply使用
def calculate_high_sales_ratio_slow(df):
    """各地域で売上1200超の割合を計算（遅い方法）"""
    return df.groupby("region")["sales"].apply(
        lambda x: (x > 1200).mean()  # 各グループに対してPython関数を実行
    )
# 速度を表示
start = time.perf_counter()
result_slow = calculate_high_sales_ratio_slow(df_large)
time_slow = time.perf_counter() - start
print(f"apply使用: {time_slow:.3f}秒")

apply使用: 0.493秒


### ベクトル化

In [21]:
# ベクトル化
def calculate_high_sales_ratio_fast(df):
    # まず、売上が1200を超えるかどうかの列を作る
    df_with_flag = df.copy()
    df_with_flag['is_high_sales'] = df_with_flag["sales"] > 1200
    
    # True/Falseの平均を取ると、Trueの割合が計算できる
    return df_with_flag.groupby("region")["is_high_sales"].mean()
# 速度を表示
start = time.perf_counter()
result_fast = calculate_high_sales_ratio_fast(df_large)
time_fast = time.perf_counter() - start
print(f"ベクトル化: {time_fast:.3f}秒")
print(f"高速化率: {time_slow/time_fast:.1f}倍")

ベクトル化: 0.388秒
高速化率: 1.3倍


あまり速くならない？？

## Named Aggregation

In [24]:
## 悪い例

# 準備：数量（qty）列も追加したデータを作成
df_multi = pd.DataFrame({
    "region": np.random.choice(regions, size=N),
    "channel": np.random.choice(channels, size=N),
    "sales": np.random.normal(1000, 200, size=N).round(2),
    "qty": np.random.randint(1, 5, size=N)  # 数量
})
# 非効率な方法：同じgroupbyを4回も実行
start = time.perf_counter()
sum_df = df_multi.groupby(["region", "channel"])['sales'].sum().rename('sales_sum')
mean_df = df_multi.groupby(["region", "channel"])['sales'].mean().rename('sales_mean')
qty_df = df_multi.groupby(["region", "channel"])['qty'].sum().rename('qty_sum')
size_df = df_multi.groupby(["region", "channel"]).size().rename('n')
# 結果を結合
result_bad = pd.concat([sum_df, mean_df, qty_df, size_df], axis=1).reset_index()
# 速度を表示
time_bad = time.perf_counter() - start
print(f"非効率な方法: {time_bad:.3f}秒")

非効率な方法: 5.465秒


In [25]:
# 効率的な方法：Named Aggregationで1回のgroupbyで完結
start = time.perf_counter()
result_good = (
    df_multi.groupby(["region", "channel"], observed=True, sort=False)
    .agg(
        sales_sum=("sales", "sum"),      # 売上の合計を sales_sum という名前で
        sales_mean=("sales", "mean"),    # 売上の平均を sales_mean という名前で
        qty_sum=("qty", "sum"),          # 数量の合計を qty_sum という名前で
        n=("sales", "size"),             # 件数を n という名前で
    )
    .reset_index()
)
# 速度を表示
time_good = time.perf_counter() - start
print(f"効率的な方法: {time_good:.3f}秒")
print(f"高速化率: {time_bad/time_good:.1f}倍")

効率的な方法: 1.615秒
高速化率: 3.4倍


In [22]:
print(pd.__version__)

1.5.3
